In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from os import listdir
from os.path import join
from sklearn import metrics
init_notebook_mode()

In [ ]:
def visualisation():
    plt.plot(y_test[:,0],label='Actual')
    plt.plot(y_pred[:,0],label='Forecast')
    plt.legend(loc="upper left")
    return plt.show

In [ ]:
t=np.arange(0,10,0.01)
mu, sigma = 0, 0.3 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
x=(np.sin(2*np.pi*t)+s)

In [ ]:
plt.plot(x)

In [ ]:
def data_prep(data,num_periods=90,f_horizon=1,inputs=1,output=1, tr_unit=901):
    data_train=data[0:tr_unit]
    x_train=data_train[:(len(data_train)-(len(data_train)%num_periods))]
    y_train=data_train[f_horizon:(len(data_train)-(len(data_train)%num_periods)+f_horizon)]
    x_batches=x_train.reshape(-1,num_periods,inputs)
    y_batches=y_train.reshape(-1,num_periods,output)
    data_test=data[tr_unit:len(data)]
    x_test=data_test[:(len(data_test)-(len(data_test)%num_periods))]
    y_test=data_test[f_horizon:(len(data_test)-(len(data_test)%num_periods)+f_horizon)]
    x_test=x_test.reshape(-1,num_periods,inputs)
    y_test=y_test.reshape(-1,num_periods,output)
    return x_batches, y_batches, x_test, y_test

In [ ]:
x_batches, y_batches, x_test, y_test=data_prep(x)

In [ ]:
np.shape(y_batches)


In [ ]:
def build_graph_RNN(data,epochs,num_periods=90,hidden=10,output=1,inputs=1,learning_rat=0.001):
    tf.reset_default_graph()
    X=tf.placeholder(tf.float32,[None,num_periods,inputs])
    Y=tf.placeholder(tf.float32,[None,num_periods,output])
    basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden)
    rnn_output, states=tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)
    stacked_rnn_output=tf.reshape(rnn_output,[-1,hidden])
    stacked_outputs=tf.layers.dense(stacked_rnn_output,output)
    outputs=tf.reshape(stacked_outputs,[-1,num_periods,output])
    loss=tf.reduce_mean(tf.squared_difference(outputs, Y))
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rat)
    training_op=optimizer.minimize(loss)
    init=tf.global_variables_initializer()
    with tf.Session() as sess:
        init.run()
        for ep in range(epochs):
            sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
            if ep % 100 == 0:
                mse=loss.eval(feed_dict={X: x_batches, Y: y_batches})
                print(ep,"\tMSE:",mse)
        y_pred=sess.run(outputs,feed_dict={X:x_test})
        mse_test=loss.eval(feed_dict={X:x_test,Y:y_test})
    print(mse, mse_test)
    return y_pred

In [ ]:
y_pred=build_graph_RNN(x,2000)

In [ ]:
plt.plot(y_test[0])
plt.plot(y_pred[0])

In [ ]:
#MSE of ultra basic model
print(metrics.mean_squared_error(x_batches[0],np.full(len(x_batches[0]),np.mean(x_batches[0]))))
#Mse of basic model ???
x_t1=y_test[0][:89]
x_t=y_test[0][1:]
print(metrics.mean_squared_error(x_t,x_t1))


In [ ]:
#MSE of predicted model
metrics.mean_squared_error(y_test[0],y_pred[0])

In [ ]:
#training data
data_train=x[0:902]
num_periods=2
x_train=np.repeat(data_train,2)
x_train1=x_train[1:1801]
x_batches=x_train1.reshape(-1,1,2)
#x_batches=x_train1.reshape(-1,2,1) this does not work
y_train=data_train[2:(len(data_train)-(len(data_train)%1)+2)]
y_batches=y_train.reshape(-1,1,1)

In [ ]:
#test data
data_test=x[902:]
x_t=np.repeat(data_test,2)
x_t1=x_t[1:193]
x_test=x_t1.reshape(-1,1,2)
#x_test=x_t1.reshape(-1,2,1)
y_t=data_test[2:(len(data_test)-(len(data_test)%1)+1)]
y_test=y_t.reshape(-1,1,1)

In [ ]:
tf.reset_default_graph()
hidden=10
X=tf.placeholder(tf.float32,[None,1,2])
Y=tf.placeholder(tf.float32,[None,1,1])
basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden,activation=tf.nn.relu)
rnn_output, states=tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)
learning_rat=0.001
stacked_rnn_output=tf.reshape(rnn_output,[-1,hidden])
stacked_outputs=tf.layers.dense(stacked_rnn_output,1)
outputs=tf.reshape(stacked_outputs,[-1,1,1])
loss=tf.reduce_mean(tf.squared_difference(outputs, Y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rat)
training_op=optimizer.minimize(loss)
init=tf.global_variables_initializer()

In [ ]:
epochs=2000
with tf.Session() as sess:
    init.run()
    for ep in range(epochs):
        sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
        if ep % 100 == 0:
            mse=loss.eval(feed_dict={X: x_batches, Y: y_batches})
            print(ep,"\tMSE:",mse)
    y_pred=sess.run(outputs,feed_dict={X:x_test})
    mse_test=loss.eval(feed_dict={X:x_test,Y:y_test})
    #NMSE=(mse/len(x_train))/np.var(x_train)
    #NMSE_test=(mse_test/len(x_test.reshape(-1)))/np.var(x_test.reshape(-1))
    print(mse)
    print(mse_test)
sess.close() 

In [ ]:
plt.plot(y_pred[:,0])
plt.plot(y_test[:,0])

In [ ]:
data_train=x[:901]
x_train=[]
for i in range(898):
    x_train.append(data_train[i:(i+3)])
x_train=np.array(x_train)
x_batches=x_train.reshape(-1,1,3)
y_train=data_train[3:len(data_train)]#+3]
y_batches=y_train.reshape(-1,1,1)
data_test=x[902:]
x_t=[]
for i in range((len(data_test)-1)-2):
    x_t.append(data_test[i:(i+3)])
x_t=np.array(x_t)
x_test=x_t.reshape(-1,1,3)
y_t=data_test[3:len(data_test)]
y_test=y_t.reshape(-1,1,1)
    


In [ ]:
np.shape(x_batches)

In [ ]:
tf.reset_default_graph()
hidden=150
X=tf.placeholder(tf.float32,[None,1,3])
Y=tf.placeholder(tf.float32,[None,1,1])
basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden,activation=tf.nn.relu)
rnn_output, states=tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)
learning_rat=0.0001
stacked_rnn_output=tf.reshape(rnn_output,[-1,hidden])
stacked_outputs=tf.layers.dense(stacked_rnn_output,1)
outputs=tf.reshape(stacked_outputs,[-1,1,1])
loss=tf.reduce_mean(tf.squared_difference(outputs, Y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rat)
training_op=optimizer.minimize(loss)
init=tf.global_variables_initializer()

In [ ]:
epochs=1500
with tf.Session() as sess:
    init.run()
    for ep in range(epochs):
        sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
        if ep % 100 == 0:
            mse=loss.eval(feed_dict={X: x_batches, Y: y_batches})
            print(ep,"\tMSE:",mse)
    y_pred=sess.run(outputs,feed_dict={X:x_test})
    mse_test=loss.eval(feed_dict={X:x_test,Y:y_test})
    #NMSE=(mse/len(x_train))/np.var(x_train)
    #NMSE_test=(mse_test/len(x_test.reshape(-1)))/np.var(x_test.reshape(-1))
    print(mse)
    print(mse_test)
sess.close() 

In [ ]:
def data_preparation(data,time_wind,batch_size=1,output=1):
    data_train=data[:901]
    x_train=[]
    for i in range((len(data_train)-1)-(time_wind-1)):
        x_train.append(data_train[i:(i+time_wind)])
    x_train=np.array(x_train)
    x_batches=x_train.reshape(-1,batch_size,time_wind)
    y_train=data_train[time_wind:len(data_train)]#+3]
    y_batches=y_train.reshape(-1,batch_size,output)
    data_test=data[902:]
    x_t=[]
    for i in range((len(data_test)-1)-(time_wind-1)):
        x_t.append(data_test[i:(i+time_wind)])
    x_t=np.array(x_t)
    x_test=x_t.reshape(-1,batch_size,time_wind)
    y_t=data_test[time_wind:len(data_test)]
    y_test=y_t.reshape(-1,batch_size,output)
    return x_batches, y_batches, x_test, y_test

    

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,3)

In [ ]:
def run_graph(epochs,time_wind,hidden=150,learning_rat=0.0065,output=1,batch_size=1):
    tf.reset_default_graph()
    X=tf.placeholder(tf.float32,[None,batch_size,time_wind])
    Y=tf.placeholder(tf.float32,[None,batch_size,output])
    basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden,activation=tf.nn.relu)
    rnn_output, states=tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)
    stacked_rnn_output=tf.reshape(rnn_output,[-1,hidden])
    stacked_outputs=tf.layers.dense(stacked_rnn_output,output)
    outputs=tf.reshape(stacked_outputs,[-1,batch_size,output])
    loss=tf.reduce_mean(tf.squared_difference(outputs, Y))
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rat)
    training_op=optimizer.minimize(loss)
    
    init=tf.global_variables_initializer()
    with tf.Session() as sess:
        init.run()
        for ep in range(epochs):
            sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
            if ep % 100 == 0:
                mse=loss.eval(feed_dict={X: x_batches, Y: y_batches})
                print(ep,"\tMSE:",mse)
        y_pred=sess.run(outputs,feed_dict={X:x_test})
        mse_test=loss.eval(feed_dict={X:x_test,Y:y_test})
    #NMSE=(mse/len(x_train))/np.var(x_train)
    #NMSE_test=(mse_test/len(x_test.reshape(-1)))/np.var(x_test.reshape(-1))
        print(mse)
        print(mse_test)
    sess.close() 
    return y_pred

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,3)
y_pred=run_graph(2000,3,hidden=10,learning_rat=0.001)
visualisation()

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,4)
y_pred=run_graph(2000,4,hidden=10,learning_rat=0.01)
visualisation()

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,10)
y_pred=run_graph(2000,10,hidden=10,learning_rat=0.01)
visualisation()

In [ ]:
array=np.array([0.0,0.1,0.15,0.22,0.35,1.0,1.1,0.9])

In [ ]:
d=[]
for i in range(6):
    d.append(array[i:(i+3)])

In [ ]:
d

In [ ]:
np.array(d)

In [ ]:
len(array)

In [ ]:
a=np.arange(0,10)
b=[]
for i in range(8):
    b.append(a[i:(i+3)])
    

In [ ]:
a=np.arange(0,10)
b=[]
for i in range(7):
    b.append(a[i:(i+4)])

In [ ]:
b

In [ ]:
t=np.arange(0,10,0.01)
mu, sigma = 0, 0.5 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
x=(t*(np.sin(2*np.pi*t))+s)

In [ ]:
plt.plot(x)

In [ ]:
x_batches, y_batches, x_test, y_test=data_prep(x)

In [ ]:
y_pred=build_graph_RNN(x,2000)

In [ ]:
plt.plot(y_test[0])
plt.plot(y_pred[0])

In [ ]:
#MSE of ultra basic model
print(metrics.mean_squared_error(x_batches[0],np.full(len(x_batches[0]),np.mean(x_batches[0]))))
#Mse of basic model ???
x_t1=y_test[0][:89]
x_t=y_test[0][1:]
print(metrics.mean_squared_error(x_t,x_t1))

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,3)
y_pred=run_graph(2000,3,hidden=10,learning_rat=0.01)
visualisation()

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,6)
y_pred=run_graph(2000,6,hidden=10,learning_rat=0.01)
visualisation()

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,10)
y_pred=run_graph(2000,10,hidden=10,learning_rat=0.01)
visualisation()

In [ ]:
t=np.arange(0,10,0.01)
t1=np.arange(0,20,0.02)
mu, sigma = 0, 0.2 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
x=(np.cos(2*np.pi*t1)*(np.sin(2*np.pi*t))+s)

In [ ]:
plt.plot(x)

In [ ]:
def data_prep(data,num_periods=90,f_horizon=1,inputs=1,output=1, tr_unit=901):
    data_train=data[0:tr_unit]
    x_train=data_train[:(len(data_train)-(len(data_train)%num_periods))]
    y_train=data_train[f_horizon:(len(data_train)-(len(data_train)%num_periods)+f_horizon)]
    x_batches=x_train.reshape(-1,num_periods,inputs)
    y_batches=y_train.reshape(-1,num_periods,output)
    data_test=data[tr_unit:len(data)]
    x_test=data_test[:(len(data_test)-(len(data_test)%num_periods))]
    y_test=data_test[f_horizon:(len(data_test)-(len(data_test)%num_periods)+f_horizon)]
    x_test=x_test.reshape(-1,num_periods,inputs)
    y_test=y_test.reshape(-1,num_periods,output)
    return x_batches, y_batches, x_test, y_test

In [ ]:
x_batches, y_batches, x_test, y_test=data_prep(x)

In [ ]:
y_pred=build_graph_RNN(x,1500,num_periods=90)

In [ ]:
plt.plot(y_test[0])
plt.plot(y_pred[0])

In [ ]:
len(y_batches[0])

In [ ]:
print(metrics.mean_squared_error(x_batches[0],np.full(len(x_batches[0]),np.mean(x_batches[0]))))
#Mse of basic model ???
x_t1=y_test[0][:89]
x_t=y_test[0][1:]
print(metrics.mean_squared_error(x_t,x_t1))

In [ ]:
def data_preparation(data,time_wind,batch_size=1,output=1):
    data_train=data[:901]
    x_train=[]
    for i in range((len(data_train)-1)-(time_wind-1)):
        x_train.append(data_train[i:(i+time_wind)])
    x_train=np.array(x_train)
    x_batches=x_train.reshape(-1,batch_size,time_wind)
    y_train=data_train[time_wind:len(data_train)]#+3]
    y_batches=y_train.reshape(-1,batch_size,output)
    data_test=data[902:]
    x_t=[]
    for i in range((len(data_test)-1)-(time_wind-1)):
        x_t.append(data_test[i:(i+time_wind)])
    x_t=np.array(x_t)
    x_test=x_t.reshape(-1,batch_size,time_wind)
    y_t=data_test[time_wind:len(data_test)]
    y_test=y_t.reshape(-1,batch_size,output)
    return x_batches, y_batches, x_test, y_test

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,3)

In [ ]:
np.shape(y_batches)

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,3)
y_pred=run_graph(2000,3,hidden=10,learning_rat=0.01)
visualisation()

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,6)
y_pred=run_graph(2000,6,hidden=10,learning_rat=0.01)
visualisation()

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,10)
y_pred=run_graph(2000,10,hidden=10,learning_rat=0.01)
visualisation()

In [ ]:
t=np.arange(0,9,0.01)
t1=np.arange(0,18,0.02)
mu, sigma = 0, 0.2 # mean and standard deviation
s = np.random.normal(mu, sigma, 900)
x1=(np.cos(2*np.pi*t1)*(np.sin(2*np.pi*t))+s)
t_=np.arange(9,10,0.01)
t1_=np.arange(18,20,0.02)
x2=np.cos(2*np.pi*t1_)*(np.sin(2*np.pi*t_))
x=np.concatenate([x1,x2])

In [ ]:
plt.plot(x)

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,1)
y_pred=run_graph(2000,1,hidden=8,learning_rat=0.01)
visualisation()

In [ ]:
print(metrics.mean_squared_error(x_batches[0],np.full(len(x_batches[0]),np.mean(x_batches[0]))))
#Mse of basic model ???
x_t1=y_test[0][:89]
x_t=y_test[0][1:]
print(metrics.mean_squared_error(x_t,x_t1))

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,3)
y_pred=run_graph(2000,3,hidden=8,learning_rat=0.01)
visualisation()

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,6)
y_pred=run_graph(2000,6,hidden=8,learning_rat=0.01)
visualisation()

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,10)
y_pred=run_graph(2000,10,hidden=5,learning_rat=0.04)
visualisation()

In [ ]:
x_batches, y_batches, x_test, y_test=data_preparation(x,15)
y_pred=run_graph(2000,15,hidden=5,learning_rat=0.08)
visualisation()

In [ ]:
x_batches,y_batches,x_test, y_test=data_preparation(x,1)

In [ ]:
np.shape(y_test)

In [ ]:
len(y_test)